# Pipeline for High-z Radio Galaxies 12: Create lists (files) with radio AGN candidates

## Introduction

In this file, three models will be applied consecutively in order to predict  
the detection of Radio Galaxies (radio AGN) and their redshift.  

In principle, this pipeline should be applied to data in Stripe 82. But  
it can be used with any other suitable dataset.

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as mpe
import matplotlib.patches as mpatches
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from astropy.visualization import LogStretch, PowerStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay
import sklearn.pipeline
import colorcet as cc
from pycaret import classification as pyc
from pycaret import regression as pyr
from joblib import dump, load
import pandas as pd
import mpl_scatter_density
import global_variables as gv
import global_functions as gf

In [2]:
mpl.rcdefaults()

---

## Reading data

Flags.

In [3]:
save_plot_flag      = False
save_preds_flag     = False
load_models_flag    = True
predict_only_hi_z   = False
use_zeroth_model    = False
use_second_z_model  = False  # z >= 3.6 (with SMOGN), or, if needed, z >= 4.0

In [4]:
used_area           = 'S82'  # can be 'S82', 'HETDEX', 'COSMOS'

In [5]:
file_name_dict      = {'S82': gv.file_S82, 'HETDEX': gv.file_HETDEX, 'COSMOS': gv.file_COSMOS}
file_name           = file_name_dict[used_area]

In [6]:
feats_2_disc_S82    = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect']
feats_2_disc_HETDEX = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect']
feats_2_disc_COSMOS = ['objID', 'RA_MILLI', 'DEC_MILLI', 'W1mag', 'W2mag', 'num_imputed', 'radio_detect', ]

feats_2_disc        = {'S82': feats_2_disc_S82, 'HETDEX': feats_2_disc_HETDEX, 'COSMOS': feats_2_disc_COSMOS}
features_2_discard  = feats_2_disc[used_area]

In [7]:
full_catalog_df     = pd.read_hdf(gv.cat_path + file_name, key='df').drop(columns=features_2_discard)

In [8]:
if used_area == 'S82':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'VLAS82_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['VLAS82_detect'])
if used_area == 'COSMOS':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'COSMOSVLA3_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['COSMOSVLA3_detect'])

Create features with class and combined redshift.

In [9]:
full_catalog_df['class']            = full_catalog_df.loc[:, 'is_AGN'].copy()
filter_non_confirmed                = np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) | np.array(full_catalog_df.loc[:, 'is_gal'] == 1)
full_catalog_df.loc[~filter_non_confirmed, 'class'] = np.nan
idx_non_Z                           = full_catalog_df.loc[:, 'Z'].where(full_catalog_df.loc[:, 'Z'] > 0).isna()
full_catalog_df.loc[idx_non_Z, 'Z'] = full_catalog_df.loc[:, 'Z'].mask(idx_non_Z, full_catalog_df.loc[idx_non_Z, 'zsp'])

Create column for detection as Radio AGN

In [10]:
full_catalog_df['radio_AGN']      = (np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) &\
                                     np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)).astype(int)

Discard minor features.

In [11]:
full_catalog_df                     = full_catalog_df.drop(columns=['is_AGN', 'is_SDSS_QSO', 'is_SDSS_gal', 'is_gal', 'zsp'])

#### As we want to predict, only use sources that have not previous spectroscopic classification

In [12]:
print(f'The used data set (in {used_area}) has {len(full_catalog_df):,} sources.')
print(f'And {np.sum(filter_non_confirmed):,} have previous spectroscopic classification.')

The used data set (in S82) has 3,590,306 sources.
And 25,689 have previous spectroscopic classification.


In [13]:
# full_catalog_df                     = full_catalog_df.loc[~filter_non_confirmed]

In [14]:
print(f'This pipeline will predict properties for {len(full_catalog_df):,} sources in {used_area}.')

This pipeline will predict properties for 3,590,306 sources in S82.


---

### Load models

In [15]:
if load_models_flag:
    if use_zeroth_model:
        star_clf        = pyc.load_model(gv.models_path + gv.star_model)  # star/no-star model
        cal_star_clf    = load(gv.models_path + gv.cal_str_model)  # calibrated model
    AGN_gal_clf         = pyc.load_model(gv.models_path + gv.AGN_gal_model)  #
    cal_AGN_gal_clf     = load(gv.models_path + gv.cal_AGN_gal_model)  # calibrated model
    radio_det_clf       = pyc.load_model(gv.models_path + gv.radio_model)  # without predicted AGN
    cal_radio_det_clf   = load(gv.models_path + gv.cal_radio_model)  # calibrated model
    redshift_reg        = pyr.load_model(gv.models_path + gv.full_z_model)  # to use on full sample
    if use_second_z_model:
        redshift_reg_2  = pyr.load_model(gv.models_path + gv.high_z_model)  # sources with predicted z >= 3.6

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [16]:
full_catalog_df = pyc.predict_model(AGN_gal_clf, data=full_catalog_df, probability_threshold=gv.AGN_thresh, raw_score=True, round=10)
full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_class', 'Score_1': 'Score_AGN'})
full_catalog_df['Score_AGN'] = np.around(full_catalog_df.loc[:, 'Score_AGN'], decimals=7)
pred_probs_AGN  = cal_AGN_gal_clf.predict(full_catalog_df.loc[:, 'Score_AGN'])
full_catalog_df['Prob_AGN'] = pred_probs_AGN

In [17]:
full_catalog_df = pyc.predict_model(radio_det_clf, data=full_catalog_df, probability_threshold=gv.radio_thresh, raw_score=True, round=10)
full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_radio', 'Score_1': 'Score_radio'})
full_catalog_df['Score_radio'] = np.around(full_catalog_df.loc[:, 'Score_radio'], decimals=7)
pred_probs_rad  = cal_radio_det_clf.predict(full_catalog_df.loc[:, 'Score_radio'])
full_catalog_df['Prob_radio'] = pred_probs_rad

In [18]:
full_catalog_df = pyr.predict_model(redshift_reg, data=full_catalog_df, round=8)
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_Z'})
full_catalog_df['pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [19]:
redshift_tol    = 0.0
if use_second_z_model:
    full_catalog_df = pyr.predict_model(redshift_reg_2, data=full_catalog_df, round=6)
    filter_pred_z   = full_catalog_df.loc[:, 'pred_Z'] >= (gv.high_z_limit + redshift_tol)
    full_catalog_df.loc[:, 'pred_Z'] = full_catalog_df.loc[:, 'pred_Z'].mask(filter_pred_z, full_catalog_df.loc[filter_pred_z, 'Label'])
    full_catalog_df = full_catalog_df.drop(columns=['Label'])
    full_catalog_df.loc[:, 'pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [20]:
full_catalog_df['pred_prob_class']    = (full_catalog_df.loc[:, 'Prob_AGN']   >= gv.cal_AGN_thresh).astype(int)
full_catalog_df['pred_prob_radio']    = (full_catalog_df.loc[:, 'Prob_radio'] >= gv.cal_radio_thresh).astype(int)

In [21]:
full_catalog_df['pred_radio_AGN']     = (np.array(full_catalog_df.loc[:, 'pred_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_radio'] == 1)).astype(int)
full_catalog_df['Score_rAGN']         = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'Score_radio']
full_catalog_df['pred_prob_rAGN']     = (np.array(full_catalog_df.loc[:, 'pred_prob_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_prob_radio'] == 1)).astype(int)
full_catalog_df['Prob_rAGN']          = full_catalog_df.loc[:, 'Prob_AGN'] * full_catalog_df.loc[:, 'Prob_radio']

rad_score_scaler                      = MinMaxScaler()
full_catalog_df['scaled_score_radio'] = rad_score_scaler.fit_transform(full_catalog_df.loc[:, 'Score_radio'].values.reshape(-1, 1))
full_catalog_df['scaled_score_rAGN']  = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'scaled_score_radio']

In [22]:
full_catalog_df.loc[:, ['class', 'pred_prob_class', 'LOFAR_detect', 'pred_prob_radio', 'Z', 'pred_Z']].describe()

,class,pred_prob_class,LOFAR_detect,pred_prob_radio,Z,pred_Z
count,25689.000000,3.590306e+06,3.590306e+06,3.590306e+06,27189.000000,3.590306e+06
mean,0.796294,1.271733e-01,4.351997e-03,1.109669e-01,1.333979,2.040464e+00
std,0.402761,3.331670e-01,6.582597e-02,3.140911e-01,0.863897,6.375727e-01
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.009090,2.000000e-02
25%,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.695000,1.781000e+00
50%,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.235000,2.178000e+00
75%,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.895000,2.178000e+00
max,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,7.011240,4.103000e+00


Obtain intermediate metrics

In [23]:
filter_radio_AGN_t      = np.array(full_catalog_df.loc[:, 'class'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)
filter_gal_AGN_t        = np.array(full_catalog_df.loc[:, 'class'] == 0) | np.array(full_catalog_df.loc[:, 'class'] == 1)
total_size              = len(full_catalog_df)
filter_AGN_t            = np.array(full_catalog_df.loc[:, 'class'] == 1)
num_AGN_t               = np.sum(filter_AGN_t)
num_gal_t               = np.sum(np.array(full_catalog_df.loc[:, 'class'] == 0))
num_radio_t             = np.sum(np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1))
num_radio_AGN_t         = np.sum(filter_radio_AGN_t)

In [24]:
# filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'Score_AGN']          >= threshold_AGN) &\
#                             np.array(full_catalog_df.loc[:, 'Score_radio']      >= threshold_radio)
filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'pred_prob_class']      == 1) &\
                            np.array(full_catalog_df.loc[:, 'pred_prob_radio']    == 1)
filt_hiz_rAGN_p         = filter_radio_AGN_p * np.array(full_catalog_df.loc[:, 'pred_Z'] >= gv.high_z_limit)
filter_AGN_p            = np.array(full_catalog_df.loc[:, 'pred_prob_class']      == 1)
filter_radio_p          = np.array(full_catalog_df.loc[:, 'pred_prob_radio']      == 1)
num_AGN_p               = np.sum(filter_AGN_p)
num_gal_p               = np.sum(np.array(full_catalog_df.loc[:, 'pred_prob_class'] == 0))
num_radio_p             = np.sum(filter_radio_p)
num_radio_AGN_p         = np.sum(filter_radio_AGN_p)

### Select sources predicted to be Radio AGN (optional)

In [25]:
# full_catalog_df         = full_catalog_df.loc[filter_radio_AGN_p]

In [26]:
full_catalog_df.loc[:, ['class', 'pred_prob_class', 'LOFAR_detect', 'pred_prob_radio', 'Z', 'pred_Z']].describe()

,class,pred_prob_class,LOFAR_detect,pred_prob_radio,Z,pred_Z
count,25689.000000,3.590306e+06,3.590306e+06,3.590306e+06,27189.000000,3.590306e+06
mean,0.796294,1.271733e-01,4.351997e-03,1.109669e-01,1.333979,2.040464e+00
std,0.402761,3.331670e-01,6.582597e-02,3.140911e-01,0.863897,6.375727e-01
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-0.009090,2.000000e-02
25%,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.695000,1.781000e+00
50%,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.235000,2.178000e+00
75%,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.895000,2.178000e+00
max,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,7.011240,4.103000e+00


Add individual metrics for redshift

In [27]:
full_catalog_df['Delta_z_N'] = np.around((full_catalog_df.loc[:, 'pred_Z'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

full_catalog_df['sigma_NMAD'] = np.around(1.48 * np.abs(full_catalog_df.loc[:, 'pred_Z'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

Numerical summary.

In [28]:
str_0_t   = f'Out of {total_size:,} initial sources in {used_area},\n'
str_1_t   = f'{num_gal_t:,} are confirmed to be galaxies. On the other side,\n'
str_2_t   = f'{num_AGN_t:,} are confirmed to be AGN. And, from the AGN,\n'
str_3_t   = f'{num_radio_AGN_t:,} are detected in radio.'

str_0_p   = f'Out of {num_radio_AGN_t:,} initial radio-detected AGN in {used_area},\n'
str_1_p   = f'{num_gal_p:,} are predicted to be galaxies. On the other side,\n'
str_2_p   = f'{num_AGN_p:,} are predicted to be AGN. And, from the predicted AGN,\n'
str_3_p   = f'{num_radio_AGN_p:,} are predicted to be detected in radio.'

In [29]:
print('-' * 60)
print(str_0_t + str_1_t + str_2_t + str_3_t)
print('-' * 60)
print(str_0_p + str_1_p + str_2_p + str_3_p)
print('-' * 60)

------------------------------------------------------------
Out of 3,590,306 initial sources in S82,
5,233 are confirmed to be galaxies. On the other side,
20,456 are confirmed to be AGN. And, from the AGN,
940 are detected in radio.
------------------------------------------------------------
Out of 940 initial radio-detected AGN in S82,
3,133,715 are predicted to be galaxies. On the other side,
456,591 are predicted to be AGN. And, from the predicted AGN,
65,867 are predicted to be detected in radio.
------------------------------------------------------------


In [30]:
cols_4_table = show_columns = ['Name', 'RA_ICRS', 'DE_ICRS', 'TYPE', 'band_num', 'class', 'pred_class',\
                               'pred_prob_class', 'Score_AGN', 'Prob_AGN', 'LOFAR_detect', 'pred_radio',\
                               'pred_prob_radio', 'Score_radio', 'Prob_radio', 'radio_AGN', 'pred_radio_AGN',\
                               'pred_prob_rAGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z']

In [31]:
cols_4_export_S82    = ['Fint_VLAS82']
cols_4_export_HETDEX = ['Sint_LOFAR']
cols_4_export_COSMOS = ['Flux_COSMOSVLA3']

cols_4_exp_all       = {'S82': cols_4_export_S82, 'HETDEX': cols_4_export_HETDEX, 'COSMOS': cols_4_export_COSMOS}

cols_photo           = ['W1mproPM', 'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 
                        'ymag', 'Jmag', 'Hmag', 'Kmag', 'W3mag', 'W4mag']

cols_4_export        = cols_4_table + cols_4_exp_all[used_area] + cols_photo

In [32]:
filter_new_rAGN = full_catalog_df.loc[:, 'radio_AGN'] == 0
if predict_only_hi_z:
    filter_high_z   = full_catalog_df.loc[:, 'pred_Z']    >= gv.high_z_limit
elif not predict_only_hi_z:
    filter_high_z   = np.ones_like(full_catalog_df.loc[:, 'pred_Z']).astype(bool)

In [33]:
full_catalog_df.loc[:, 'TYPE'] = full_catalog_df.loc[:, 'TYPE'].mask(full_catalog_df.loc[:, 'TYPE'] == '    ', 'CCCC')

In [34]:
display(full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z'], ascending=False).head(15))

,Name,RA_ICRS,DE_ICRS,TYPE,band_num,class,pred_class,pred_prob_class,Score_AGN,Prob_AGN,LOFAR_detect,pred_radio,pred_prob_radio,Score_radio,Prob_radio,radio_AGN,pred_radio_AGN,pred_prob_rAGN,Score_rAGN,Prob_rAGN,Z,pred_Z,Fint_VLAS82,W1mproPM,W2mproPM,gmag,rmag,imag,zmag,ymag,Jmag,Hmag,Kmag,W3mag,W4mag
2684467,J020320.44+004029.9,30.835190,0.674982,,7,NaN,0,0,0.414104,0.027847,0,0,0,0.052391,0.065826,0,0,0,0.021695,0.001833,NaN,4.103,17.860001,19.597000,19.809999,22.099701,20.625700,20.257799,20.021700,19.858601,17.450001,17.24,16.590000,16.670000,14.62
3511189,J224421.39+012018.8,341.089172,1.338558,,12,NaN,0,0,0.398816,0.014283,0,0,0,0.076690,0.093915,0,0,0,0.030585,0.001341,NaN,4.102,17.860001,18.715000,18.964001,21.032900,19.525200,19.203899,18.976101,18.749901,17.450001,17.24,16.590000,16.670000,14.62
3511643,J224421.63+012020.7,341.090179,1.339097,,12,NaN,0,0,0.398816,0.014283,0,0,0,0.070643,0.087032,0,0,0,0.028174,0.001243,NaN,4.096,17.860001,18.799999,19.048000,21.032900,19.525200,19.203899,18.976101,18.749901,17.450001,17.24,16.590000,16.670000,14.62
1768417,J225047.39-000133.8,342.697510,-0.026075,,7,NaN,0,0,0.418509,0.033609,0,1,1,0.322234,0.332118,0,0,0,0.134858,0.011162,NaN,4.091,17.860001,19.548000,19.809999,23.299999,21.690701,21.365200,21.177200,21.400000,17.450001,17.24,16.590000,16.506001,14.62
38246,J230737.57-012307.5,346.906586,-1.385441,,9,NaN,0,0,0.404498,0.018353,0,1,1,0.222104,0.242999,0,0,0,0.089840,0.004460,NaN,4.089,17.860001,18.792000,19.066000,21.571400,20.179199,19.749800,19.526600,19.320400,17.450001,17.24,16.590000,16.670000,14.62
2801090,J231303.33+004546.6,348.263916,0.762957,,9,NaN,0,0,0.399303,0.014595,0,1,1,0.216301,0.237550,0,0,0,0.086370,0.003467,NaN,4.080,17.860001,18.761999,19.139000,21.470800,20.065901,19.685101,19.437901,19.336300,17.450001,17.24,16.590000,16.670000,14.62
1668095,J225830.49-000612.0,344.627106,-0.103350,,12,NaN,0,0,0.393201,0.011118,0,0,0,0.105661,0.126004,0,0,0,0.041546,0.001401,NaN,4.073,17.860001,18.117001,18.528999,20.344500,18.110001,17.665001,17.423000,17.675301,17.339001,17.02,16.582001,16.670000,14.62
3242628,J230224.27+010609.5,345.601166,1.102631,,7,NaN,0,0,0.474843,0.278366,0,0,0,0.058680,0.073208,0,0,0,0.027864,0.020379,NaN,4.071,17.860001,19.276999,19.561001,22.026800,20.572001,20.210199,20.020901,19.790001,17.450001,17.24,16.590000,16.670000,14.62
1056310,J004455.69-003401.7,11.232084,-0.567148,,8,NaN,0,0,0.403748,0.017758,0,1,1,0.432780,0.422021,0,0,0,0.174734,0.007494,NaN,4.065,17.860001,19.650999,19.809999,23.299999,20.985300,20.596100,20.272200,19.945700,17.450001,17.24,16.590000,16.670000,14.62
2311652,J004812.93+002330.6,12.053917,0.391826,,9,NaN,0,0,0.405825,0.019451,0,0,0,0.040597,0.051747,0,0,0,0.016475,0.001007,NaN,4.065,17.860001,19.476000,19.809999,22.151400,20.756001,20.373899,20.132500,19.913601,17.450001,17.24,16.590000,16.670000,14.62


In [35]:
if save_preds_flag:
    full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z'],
     ascending=False).to_csv(gv.preds_path + f'predicted_rAGN_{used_area}.csv', index_label='ID')

---

In [36]:
temp_flag = False

In [37]:
if temp_flag:
    
    add_columns  = ['band_num', 'class', 'pred_prob_class', 'Score_AGN', 'Prob_AGN', 
                    'LOFAR_detect',  'pred_prob_radio', 'Score_radio', 'Prob_radio', 
                    'radio_AGN', 'pred_prob_rAGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z']
    used_colours = ['g_r', 'g_i', 'g_J',  'r_i', 'r_z', 'r_J', 'r_W1', 'i_z', 
                    'i_y', 'z_y', 'z_W1', 'y_J', 'y_W1', 'y_W2', 'J_H', 
                    'K_W3', 'K_W4', 'H_K', 'H_W3', 'W1_W2', 'W1_W3', 'W2_W3', 'W3_W4']

In [38]:
if temp_flag:
    saving_data = full_catalog_df.loc[:, add_columns + cols_photo + used_colours]
    saving_data['ID'] = saving_data.index
    saving_data.to_hdf(gv.preds_path + f'{used_area}_for_prediction.h5', key='df')